# Build community model using top 50 ASV genera

In [1]:
import cobra
import os
import sys
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
from mb_xai import mb_utils
from functools import reduce
from os.path import join
import pandas as pd
from micom.data import test_taxonomy
from micom import Community
from micom import load_pickle
# df = reduce(lambda df1,df2: pd.merge(df1,df2,on='id'), dfList)

SAVE_OUTPUT = True
DATA_LOC = '../../../Data/microbiome_xai/'
FIG_SAVE_LOC = "../figures/"
# TABLE_SAVE_LOC = "../tables/"
TABLE_SAVE_LOC = DATA_LOC+"agp_data/"

dir_agora2 = "/mnt/d/Microbiome_project/reconstructions/AGORA2_recons/"
# dir_taxon = "../merged-data/"
dir_taxon_asv = DATA_LOC+"agp_data/taxonomy-data/"
dir_taxon_silva = DATA_LOC+"agp_data/silva_merged-data/"

In [2]:
taxon_df = pd.read_csv(TABLE_SAVE_LOC + "SILVA_taxon_df_merge_fd.csv",index_col=0)
taxon_df.fillna("",inplace=True)
asv_df = pd.read_csv(TABLE_SAVE_LOC + "asv_df_merged_fd.csv",index_col=0)

DIR_MERGED = "/mnt/d/Microbiome_project/reconstructions/merged_recons/"
number_of_genus = 70

In [3]:
genus_strain_dict, species_strain_dict, genus_species_dict = mb_utils.get_agora_strain_dicts(dir_agora2)
taxon_df, agora_notin = mb_utils.get_agora_taxon_df(taxon_df, genus_strain_dict, species_strain_dict)

414
genus: in- 99 , not in- 34
species: in- 51 , not in- 37


# Top 50 genus based on genus correlations

In [37]:
import mb_xai.gut_data as gd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler

def match_Xy_df(X_in_df, y_in_df):
    X_df, y_df = X_in_df.copy(), y_in_df.copy()
    """ Makes sure X_df and y_df have the same indices"""
    overlap = list(set(X_df.index).intersection(set(y_df.index)))
    X_df =  X_df.loc[overlap]
    y_df =  y_df.loc[overlap]
    return X_df, y_df

In [36]:
DATA_LOC = '../../../Data/microbiome_xai/'
gut_data = gd.GutData()
gut_data.load_data(
    # FILE_COMM_MODEL= DATA_LOC + 'reconstructions/community_5_TOP-vegan.pickle',
    # FILE_COMM_MODEL= DATA_LOC + 'reconstructions/community_50_TOP.pickle',
    FILE_COMM_MODEL= DATA_LOC + 'reconstructions/community_top50_fd.pickle',
    # FILE_GENUS_ASVS = DATA_LOC + 'agp_data/taxon_genus_asvs.csv',
    FILE_GENUS_ASVS = DATA_LOC + 'agp_data/SILVA_genus_counts_fd.csv',
    FILE_METADATA = DATA_LOC + "agp_data/metadata_biosample_filtered.csv",
    DIR_SIM_DATA = DATA_LOC + "micom-sim-data/"
)

Fixed EX_tDHNACOA(e)


In [38]:
input_type = "abundance"#"flux"
SAMPLE_NUM=10000
top_genus_trait_df = pd.DataFrame()
scaler = StandardScaler()
# gut_data.set_vegan_df(sample_num=10000)
for phenotype in ["vegan", "ibs", "t2d", "ibd"][:]: #  "ibs", "t2d", "ibd" # not enough samples for t2d and ibd
    
    gut_data.norm_abundances(filter_model=False, add_delta=True) ## Filters genus to those in model, adds small value to abundaces
    
    gut_data.X_df = gut_data.asv_df.T.copy()
    gut_data.sample_list = gut_data.X_df.index.to_list()

    if phenotype=="vegan":
        gut_data.set_vegan_df(sample_num=SAMPLE_NUM)
    elif phenotype=="ibs":
        gut_data.set_ibs_df(sample_num=SAMPLE_NUM, add_other_diagnosis=False)
    elif phenotype=="t2d":
        gut_data.set_t2d_df(sample_num=SAMPLE_NUM, add_other_diagnosis=False)
    elif phenotype=="ibd":
        gut_data.set_ibd_df(sample_num=SAMPLE_NUM, add_other_diagnosis=False)

    y_df = gut_data.y_df.copy()
    if input_type=="flux":
        X, y = match_Xy_df(X_flux.copy(), y_df)
    elif input_type=="abundance":
        X, y = match_Xy_df(gut_data.X_df.copy(), y_df)

    X_scaled = scaler.fit_transform(X)
    f_stat, p_val = f_classif(X_scaled, y)
    #fclass_df = pd.DataFrame(f_stat, index=X.columns, columns=["f_stat"])
    fclass_pval_df = pd.Series(p_val, index=X.columns)
    fclass_pval_df.name = phenotype+"_pval"
    # fclass_df = pd.DataFrame([f_stat, p_val], index=X.columns, columns=["f_stat", "p_val"])
    fclass_pval_df.sort_values(inplace=True)
    top_genus_trait_df = top_genus_trait_df.append(fclass_pval_df)
top_genus_trait_df

... normalizing raw ASV abundances to fractions, dropping samples with 0 total abundances
... normalizing raw ASV abundances to fractions, dropping samples with 0 total abundances
... normalizing raw ASV abundances to fractions, dropping samples with 0 total abundances
... normalizing raw ASV abundances to fractions, dropping samples with 0 total abundances


,clostridioides,butyricicoccus,gastranaerophilales,coprococcus,roseburia,alistipes,nk4a214,lachnospira,anaerosporobacter,incertae,...,oceanobacillus,dialister,finegoldia,succinivibrio,subdoligranulum,campylobacter,bifidobacterium,clostridia,erysipelotrichaceae,corynebacterium
vegan_pval,9.078560e-11,2.022733e-07,1.029121e-06,0.000009,0.000040,1.277447e-04,0.000192,0.000704,0.000838,0.001175,...,0.831148,0.858701,0.883389,0.898699,0.916299,0.932388,0.937408,0.956584,0.966356,0.972753
ibs_pval,2.992389e-01,9.561707e-01,4.658989e-02,0.004952,0.572803,6.630175e-01,0.543036,0.632272,0.225013,0.000335,...,0.684519,0.181070,0.432575,0.082110,0.075956,0.291619,0.028280,0.580855,0.019602,0.606751
t2d_pval,5.082514e-01,5.217414e-01,2.053190e-01,0.588795,0.377321,9.956771e-02,0.986304,0.502977,0.035211,0.000010,...,0.845416,0.090250,0.315521,0.462610,0.969854,0.450265,0.275823,0.743302,0.027491,0.160934
ibd_pval,5.769198e-01,1.721085e-01,4.506741e-07,0.000013,0.104849,1.412603e-09,0.256671,0.013465,0.000325,0.737900,...,0.763636,0.352254,0.101232,0.220165,0.061811,0.258490,0.071940,0.135741,0.644295,0.375727


In [39]:
agora2_genus_models = [x.split(".json")[0] for x in os.listdir(DIR_MERGED)]

top_genus_trait_df = top_genus_trait_df.T.copy()
top_genus_trait_df["agora2"] = top_genus_trait_df.index.map(lambda x: True if x in agora2_genus_models else False)

### Add column to note if strain is in top 50
asv_count_threshold = 0
genus_ASV = mb_utils.group_asvs(taxon_df,asv_df, grp_id="genus", asv_count_threshold=asv_count_threshold)
common_genus = genus_ASV.loc[genus_ASV.index.notnull()].sum(axis=1).sort_values(
    ascending=False
)[:number_of_genus].index.tolist()
models_2_join, topX_genus = [],[]
for mod in os.listdir(DIR_MERGED):
    mod_name = mod.split(".json")[0]
    if mod_name in genus_species_dict.keys() and mod_name in common_genus:
        models_2_join.append(mod)
        topX_genus.append(mod_name)
print("# of models to build into community:",len(models_2_join))
print(len(topX_genus),len(common_genus))

top_genus_trait_df["agora2_top50"] = top_genus_trait_df.index.map(lambda x: True if x in topX_genus else False)
top_genus_trait_df

(133, 9580)
(133, 9580)
# of models to build into community: 50
50 70


,vegan_pval,ibs_pval,t2d_pval,ibd_pval,agora2,agora2_top50
clostridioides,9.078560e-11,0.299239,0.508251,5.769198e-01,True,False
butyricicoccus,2.022733e-07,0.956171,0.521741,1.721085e-01,True,True
gastranaerophilales,1.029121e-06,0.046590,0.205319,4.506741e-07,False,False
coprococcus,8.794128e-06,0.004952,0.588795,1.258155e-05,True,True
roseburia,4.013211e-05,0.572803,0.377321,1.048486e-01,True,True
...,...,...,...,...,...,...
campylobacter,9.323880e-01,0.291619,0.450265,2.584899e-01,True,False
bifidobacterium,9.374076e-01,0.028280,0.275823,7.194006e-02,True,True
clostridia,9.565842e-01,0.580855,0.743302,1.357408e-01,False,False
erysipelotrichaceae,9.663557e-01,0.019602,0.027491,6.442947e-01,True,True


In [40]:
top_genus_trait_df.loc[(top_genus_trait_df["vegan_pval"]<0.05)&(top_genus_trait_df["agora2"]==True)]

,vegan_pval,ibs_pval,t2d_pval,ibd_pval,agora2,agora2_top50
clostridioides,9.078560e-11,0.299239,0.508251,5.769198e-01,True,False
butyricicoccus,2.022733e-07,0.956171,0.521741,1.721085e-01,True,True
coprococcus,8.794128e-06,0.004952,0.588795,1.258155e-05,True,True
roseburia,4.013211e-05,0.572803,0.377321,1.048486e-01,True,True
alistipes,1.277447e-04,0.663017,0.099568,1.412603e-09,True,True
lachnospira,7.044397e-04,0.632272,0.502977,1.346469e-02,True,True
anaerosporobacter,8.382146e-04,0.225013,0.035211,3.246546e-04,True,False
marvinbryantia,1.883829e-03,0.732529,0.138503,2.544059e-01,True,False
bilophila,3.382375e-03,0.050279,0.210831,1.488198e-01,True,True
flavonifractor,8.151045e-03,0.119635,0.319899,2.865852e-01,True,True


#### Do it based on ML feat importance


In [78]:
SAVE_ID = "stdscale_50feats_c1"#"stdscale", "maxmin" # "minmax"
imp_feat_abundance_pheno_df = pd.read_csv(gut_data.dir_sim_data+'imp_feat_abundance_%s_fd.csv'%(SAVE_ID), index_col=0)

###
agora2_genus_models = [x.split(".json")[0] for x in os.listdir(DIR_MERGED)]

top_genus_trait_df = imp_feat_abundance_pheno_df.copy()
top_genus_trait_df["agora2"] = top_genus_trait_df.index.map(lambda x: True if x in agora2_genus_models else False)

### Add column to note if strain is in top 50
asv_count_threshold = 0
genus_ASV = mb_utils.group_asvs(taxon_df,asv_df, grp_id="genus", asv_count_threshold=asv_count_threshold)
common_genus = genus_ASV.loc[genus_ASV.index.notnull()].sum(axis=1).sort_values(
    ascending=False
)[:number_of_genus].index.tolist()
models_2_join, topX_genus = [],[]
for mod in os.listdir(DIR_MERGED):
    mod_name = mod.split(".json")[0]
    if mod_name in genus_species_dict.keys() and mod_name in common_genus:
        models_2_join.append(mod)
        topX_genus.append(mod_name)
print("# of models to build into community:",len(models_2_join))
print(len(topX_genus),len(common_genus))

top_genus_trait_df["agora2_top50"] = top_genus_trait_df.index.map(lambda x: True if x in topX_genus else False)

trait_to_community_dict = {}
for trait in imp_feat_abundance_pheno_df.columns:
    top_trait_genera_list = top_genus_trait_df.abs()[(
        top_genus_trait_df["agora2"]==True)&(
            top_genus_trait_df[trait].notnull())].sort_values(
        by=[trait],ascending=False).index
            
    top_trait_genera_list = list(top_trait_genera_list)
    print(len(top_trait_genera_list))
    top_trait_genera_list
    trait_to_community_dict.update({trait: top_trait_genera_list})
    
trait_to_community_dict

(133, 9580)
(133, 9580)
# of models to build into community: 50
50 70
34
36
36
26


{'vegan_abundance': ['erysipelatoclostridium',
  'turicibacter',
  'frisingicoccus',
  'acidaminococcus',
  'bilophila',
  'lachnoclostridium',
  'butyricimonas',
  'dorea',
  'akkermansia',
  'butyricicoccus',
  'collinsella',
  'romboutsia',
  'flavonifractor',
  'parasutterella',
  'desulfovibrio',
  'ruminococcus',
  'barnesiella',
  'oscillibacter',
  'faecalibacterium',
  'roseburia',
  'alistipes',
  'anaerosporobacter',
  'bifidobacterium',
  'bacteroides',
  'lachnospira',
  'marvinbryantia',
  'coprococcus',
  'coprobacter',
  'odoribacter',
  'fusicatenibacter',
  'parabacteroides',
  'anaeroplasma',
  'prevotella',
  'eubacterium'],
 'ibs_abundance': ['megasphaera',
  'sutterella',
  'rikenellaceae',
  'bacteroides',
  'bifidobacterium',
  'tyzzerella',
  'erysipelotrichaceae',
  'prevotella',
  'ruminococcus',
  'parasutterella',
  'odoribacter',
  'subdoligranulum',
  'methanobrevibacter',
  'megamonas',
  'fusicatenibacter',
  'anaerococcus',
  'succinivibrio',
  'staphy

In [63]:
top_genus_trait_df.abs().sort_values(by=["vegan_abundance"],ascending=False)[:10]# [(top_genus_trait_df["agora2"]==True)]

,vegan_abundance,ibs_abundance,t2d_abundance,ibd_abundance,agora2,agora2_top50
negativibacillus,1.448891,NaN,0.018445,NaN,0,0
erysipelatoclostridium,0.766281,NaN,NaN,NaN,1,0
turicibacter,0.760855,NaN,0.275781,NaN,1,0
frisingicoccus,0.694417,NaN,0.037001,0.769456,1,0
acidaminococcus,0.514403,0.020609,0.328401,NaN,1,0
incertae,0.400253,0.047128,0.275742,NaN,0,0
bilophila,0.392793,NaN,0.023495,NaN,1,1
lachnoclostridium,0.381743,NaN,0.039058,NaN,1,1
butyricimonas,0.379258,NaN,NaN,0.303239,1,0
dorea,0.378407,0.001616,NaN,NaN,1,1


# Top 50 genus based on ASV counts

['abiotrophia', 'abiotrophia_defectiva']

In [8]:
asv_count_threshold = 0
genus_ASV = mb_utils.group_asvs(taxon_df,asv_df, grp_id="genus", asv_count_threshold=asv_count_threshold)
common_genus = genus_ASV.loc[genus_ASV.index.notnull()].sum(axis=1).sort_values(
    ascending=False
)
common_genus

(133, 9580)
(133, 9580)


bacteroides          47862017.0
faecalibacterium     14758626.0
                      7766775.0
prevotella            7536076.0
eubacterium           6449274.0
                        ...    
senegalimassilia        20905.0
pseudoxanthomonas       20865.0
murdochiella            20597.0
gemella                 20253.0
varibaculum             19495.0
Length: 133, dtype: float64

In [14]:
common_genus.loc["bacillus"]

164783.0

In [6]:
asv_count_threshold = 0

genus_ASV = mb_utils.group_asvs(taxon_df,asv_df, grp_id="genus", asv_count_threshold=asv_count_threshold)

common_genus = genus_ASV.loc[genus_ASV.index.notnull()].sum(axis=1).sort_values(
    ascending=False
)[:number_of_genus].index.tolist()

models_2_join, topX_genus = [],[]
for mod in os.listdir(DIR_MERGED):
    mod_name = mod.split(".json")[0]
    if mod_name in genus_species_dict.keys() and mod_name in common_genus:
        models_2_join.append(mod)
        topX_genus.append(mod_name)
print("# of models to build into community:",len(models_2_join))
print(len(topX_genus),len(common_genus))

(133, 8807)
(133, 8807)
# of models to build into community: 50
50 70


In [11]:
taxonomy = pd.DataFrame()

for mod in models_2_join[:]:
    # print(mod)
    tax_dict = {
        "id": mod.split(".json")[0], 
        "genus": mod.split("_")[0], 
        "species": mod.split(".json")[0],
        "file": DIR_MERGED+mod
    }
    taxonomy = pd.concat([taxonomy, pd.DataFrame.from_dict(tax_dict,orient="index").T],ignore_index=True)
    
taxonomy.head()

In [13]:
taxonomy.head()

,id,genus,species,file
0,acinetobacter,acinetobacter.json,acinetobacter,/mnt/d/Microbiome_project/reconstructions/merg...
1,akkermansia,akkermansia.json,akkermansia,/mnt/d/Microbiome_project/reconstructions/merg...
2,alistipes,alistipes.json,alistipes,/mnt/d/Microbiome_project/reconstructions/merg...
3,anaerostipes,anaerostipes.json,anaerostipes,/mnt/d/Microbiome_project/reconstructions/merg...
4,bacteroides,bacteroides.json,bacteroides,/mnt/d/Microbiome_project/reconstructions/merg...


Takes about 7 minutes to build 50 genera community model (progress=False probably helps)

In [15]:
com_model = Community(taxonomy, name="gut_community_top50",progress=False)

[13:26:58] WARNING  Reaction tDHNACOA__lactobacillus seems to be an exchange ]8;id=187341;file:///home/eskavvas/anaconda3/lib/python3.8/site-packages/micom/community.py\community.py]8;;\:315
                    reaction but its ID does not start with 'EX_'...

In [16]:
save_com_model_loc = '../../../Data/microbiome_xai/reconstructions/community_top50_fd.pickle'
com_model.to_pickle(save_com_model_loc)

In [ ]:
# com_model.to_pickle("../reconstructions/community_top50_fd.pickle")

# Test community model optimizations

In [17]:
import mb_xai.gut_data as gd

In [19]:
DATA_LOC = '../../../Data/microbiome_xai/'
gut_data = gd.GutData()
gut_data.load_data(
    # FILE_COMM_MODEL='../data/reconstructions/community_5_TOP-vegan.pickle',
    # FILE_COMM_MODEL= DATA_LOC + 'reconstructions/community_50_TOP.pickle',
    FILE_COMM_MODEL= DATA_LOC + 'reconstructions/community_top50_fd.pickle',
    # FILE_GENUS_ASVS = "../data/agp_data/taxon_genus_asvs.csv",
    FILE_GENUS_ASVS = DATA_LOC + 'agp_data/taxon_genus_asvs.csv',
    # FILE_METADATA = DATA_LOC + "agp_data/mcdonald_agp_metadata.txt",
    FILE_METADATA = DATA_LOC + "agp_data/metadata_biosample_filtered.csv",
    DIR_SIM_DATA = DATA_LOC + "micom-sim-data/"  # "../data/micom-sim-data/",
)
gut_data.norm_abundances(filter_model=True, add_delta=True) ## Filters genus to those in model, adds small value to abundaces
gut_data.X_df = gut_data.asv_df.T.copy()
gut_data.sample_list = gut_data.X_df.index.to_list()
# gut_data.set_vegan_df(sample_num=40)
# ## Set vegan df changes X_df and y_df and will therefore change medium_df. Be sure to run medium df after setting samples
# medium_df = pd.DataFrame(1000, columns=gut_data.com_model.medium.keys(), index = gut_data.X_df.index)
# gut_data.sample_medium_dict = medium_df.T.to_dict()
# gut_data.return_fluxes = True
# gut_data.pfba_bool = True # otherwise optimum values will not be fluxes but intead min(sum flux)

Fixed EX_tDHNACOA(e)
... normalizing raw ASV abundances to fractions, dropping samples with 0 total abundances
# of genus in model and not filterd out of QA/QC: 50, # of genus not in model:0


In [20]:
gut_data.com_model

Name,None
Memory address,0x07facdf098a60
Number of metabolites,90619
Number of reactions,110071
Number of groups,0
Objective expression,1.0*community_objective
Compartments,"c__acinetobacter, e__acinetobacter, p__acinetobacter, m, c__akkermansia, e__akkermansia, p__akkermansia, c__alistipes, e__alistipes, p__alistipes, c__anaerostipes, e__anaerostipes, c__bacteroides, e__bacteroides, p__bacteroides, c__barnesiella, e__barnesiella, p__barnesiella, c__bifidobacterium, e__bifidobacterium, c__bilophila, e__bilophila, p__bilophila, c__blautia, e__blautia, c__butyricicoccus, e__butyricicoccus, c__butyrivibrio, e__butyrivibrio, c__christensenellaceae, e__christensenellaceae, e__clostridium, c__clostridium, c__collinsella, e__collinsella, c__comamonas, e__comamonas, p__comamonas, c__coprococcus, e__coprococcus, c__corynebacterium, e__corynebacterium, c__dialister, e__dialister, p__dialister, c__dorea, e__dorea, c__dysgonomonas, e__dysgonomonas, p__dysgonomonas, c__enterococcus, e__enterococcus, c__erysipelotrichaceae, e__erysipelotrichaceae, c__eubacterium, e__eubacterium, c__faecalibacterium, e__faecalibacterium, c__finegoldia, e__finegoldia, c__flavonifractor, e__flavonifractor, c__fusicatenibacter, e__fusicatenibacter, c__haemophilus, e__haemophilus, p__haemophilus, c__intestinibacter, e__intestinibacter, c__lachnoclostridium, e__lachnoclostridium, c__lachnospira, e__lachnospira, c__lachnospiraceae, e__lachnospiraceae, c__lactobacillus, e__lactobacillus, c__oscillibacter, e__oscillibacter, c__parabacteroides, e__parabacteroides, p__parabacteroides, c__paraprevotella, e__paraprevotella, p__paraprevotella, c__parasutterella, e__parasutterella, p__parasutterella, c__phascolarctobacterium, e__phascolarctobacterium, p__phascolarctobacterium, c__prevotella, e__prevotella, p__prevotella, c__providencia, e__providencia, p__providencia, c__pseudomonas, e__pseudomonas, p__pseudomonas, c__romboutsia, e__romboutsia, c__roseburia, e__roseburia, e__ruminococcus, c__ruminococcus, c__stenotrophomonas, e__stenotrophomonas, p__stenotrophomonas, c__streptococcus, e__streptococcus, c__subdoligranulum, e__subdoligranulum, c__sutterella, e__sutterella, p__sutterella, c__tyzzerella, e__tyzzerella, c__veillonella, e__veillonella, p__veillonella"


In [22]:
sol = gut_data.com_model.optimize()

In [23]:
sol

,abundance,growth_rate,reactions,metabolites
compartments,,,,
acinetobacter,0.02,16939.847461,2689,2346
akkermansia,0.02,157.557843,1385,1220
alistipes,0.02,2425.077576,3283,2563
anaerostipes,0.02,954.479435,1605,1279
bacteroides,0.02,34714.737723,4649,3619
...,...,...,...,...
streptococcus,0.02,105694.851055,3525,2731
subdoligranulum,0.02,62.435296,1245,1081
sutterella,0.02,143.463375,1697,1602
